# Hints

## Download data

In [ ]:
from cloud_ml.storage.api import Storage

s3 = Storage.s3(access_key='Le9tg70HQEJsoGqjqXH8', secret_key='PUT_SECRET_HERE')
s3.get_dir('dl-hse-2021/vggsound_small_contest/', './vggsound_small_contest/')

## Read video

Hints:
* Читать видео можно с помощью OpenCV: [docs](https://docs.opencv.org/master/dd/d43/tutorial_py_video_display.html)
* Чтение происходит покадрово. Пример в функции `read_full_video`
* Далеко не всегда необходимо использовать все кадры из видео. 
    Чаще всего достаточно читать несколько кадров с равным отступом друг от друга. 
    Пример в фукнции `read_video_sample`
* Разные видео могут иметь разные пространственные размерности. 
    Это проблема при формировании батчей. 
    Побороться с этим может помочь `cv2.resize(...)`
* Разные видео могут иметь разное количетсво кадров. 
    Это проблема при формировании батчей. 
    Самое простое решение – паддинг нулевыми фреймами
* `torch.nn.Conv3d` принимает особый порядок размерностей: [docs](https://pytorch.org/docs/stable/generated/torch.nn.Conv3d.html)

In [ ]:
from pathlib import Path

import cv2
import numpy as np

In [ ]:
def read_full_video(file_path: Path) -> np.ndarray:
    cap = cv2.VideoCapture(file_path.as_posix())

    frames_stack = []
    while cap.isOpened():
        ret = cap.grab()
        if not ret:
            break
        
        frame = cap.retrieve()[1]
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames_stack.append(frame)
    cap.release()
    video = np.stack(frames_stack)
    
    return video


def read_video_sample(file_path: Path,
                      num_frames_to_read: int) -> np.ndarray:
    cap = cv2.VideoCapture(file_path.as_posix())
    num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if num_frames_to_read <= num_frames:
        frames_idxs = np.linspace(0, num_frames-1, num_frames_to_read, dtype=np.int16)
        frames_idxs = set(frames_idxs)
    else:
        raise ValueError(f'Param "num_frames_to_read" ({num_frames_to_read}) must be less or equal'
                         f'than number of frames in given video ({num_frames})')

    frames_stack, cur_frame_idx = [], 0
    while cap.isOpened():
        ret = cap.grab()
        if not ret:
            break

        if cur_frame_idx in frames_idxs:
            frame = cap.retrieve()[1]
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames_stack.append(frame)
        cur_frame_idx += 1
    cap.release()
    video = np.stack(frames_stack)

    return video

## Read audio

Hints:
* Читать аудио можно с помощью librosa: [docs](https://librosa.org/doc/latest/core.html)
* С аудио можно работать как с картинкой или как с сырым сигналом.
* Внимательно посмотрите на параметры фукнции librosa.load(...). 
    Ресемплинг – это не очень дешевая операция
* Разные аудио могут иметь разные sample rate и разную продолжительность. 
    Это снова проблема для сбора батчей. 
    Это можно решить паддингами

In [ ]:
from pathlib import Path

import librosa


# Меня librosa часто заваливает предупреждениями о переключение бэкенда.
#   не лучшее решение:
# import warnings
# warnings.filterwarnings('ignore', message='PySoundFile')

In [ ]:
def read_audio(file_path: Path):
    audio, sample_rate = librosa.load(file_path)
    
    return audio


def audio_to_spectrogram(audio: np.ndarray,
                         decibel_scale: bool = True,
                         spec_kws: dict = {}) -> np.ndarray:
        spec = np.abs(librosa.stft(audio, **spec_kws))

        if decibel_scale:
            spec = librosa.amplitude_to_db(spec, ref=np.max)

        return spec